
# Leetcodeify Friend Recommendations
Write an SQL query to recommend friends to Leetcodify users. We recommend user x to user y if:

Users x and y are not friends, and
Users x and y listened to the same three or more different songs on the same day.

# SQL CODE
`````sql
with base as (
    select
        user_id
        , song_id
        , day
    from listens
    group by user_id, song_id, day
)

, base_calc as (
    select
        a.user_id as user_id 
        ,b.user_id as recommended_id
        ,a.day
        ,count(1) as match_count
        , concat(a.user_id,"|",b.user_id) as f_key
    from base a
    join base b
        on a.song_id = b.song_id
        and a.day = b.day
        and a.user_id <> b.user_id
    group by a.user_id, b.user_id, a.day
)

, exclusion as (
    select concat(u1,"|",u2) as f_key
    from
    (
        select user1_id as u1, user2_id as u2 from friendship
        union all
        select user2_id, user1_id from friendship
    ) as ex
)

select  
    user_id, recommended_id 
from base_calc 
where match_count >=3 
    and f_key not in (select f_key from exclusion) 
group by user_id, recommended_id 
`````


In [1]:
import pandas as pd

In [2]:
"""
    get leet code data. stored in json file
"""
r = open('Leetcodify Friends Recommendations Data.json','r')
input_data = eval(r.read())

r = open('Leetcodify Friends Recommendations Answer.json','r')
answer_data = eval(r.read())

In [3]:
pd_listens = pd.DataFrame(input_data['rows']['Listens'], columns=input_data['headers']['Listens'])
pd_friends = pd.DataFrame(input_data['rows']['Friendship'], columns=input_data['headers']['Friendship'])
pd_answer_sql = pd.DataFrame(answer_data['values'], columns=answer_data['headers'])

In [4]:
pd_listens

,user_id,song_id,day
0,1,10,2021-03-15
1,1,11,2021-03-15
2,1,12,2021-03-15
3,2,10,2021-03-15
4,2,11,2021-03-15
5,2,12,2021-03-15
6,3,10,2021-03-15
7,3,11,2021-03-15
8,3,12,2021-03-15
9,4,10,2021-03-15


In [5]:
pd_friends

,user1_id,user2_id
0,1,2


In [6]:
pd_answer_sql

,user_id,recommended_id
0,1,3
1,2,3
2,3,1
3,3,2


In [31]:
def concat(x,y): return '|'.join([str(x),str(y)])

pd_rec = pd_listens.merge(pd_listens, how="inner", on=['song_id','day'] , suffixes=("_1","_2") )

pd_rec = pd_rec[ pd_rec.user_id_1 != pd_rec.user_id_2 ]

pd_rec['match_count'] = pd_rec.groupby(['user_id_1', 'user_id_2', 'day'])['user_id_1'].transform('count')

pd_rec = pd_rec[['user_id_1', 'user_id_2']][ pd_rec.match_count >= 3 ]

pd_rec = pd_rec.drop_duplicates()

pd_rec.reset_index(inplace=True)

pd_rec = pd_rec.drop(columns='index')
pd_rec['friend_key'] = [k for k in map(concat, pd_rec.user_id_1, pd_rec.user_id_2)]
pd_friends_1 = pd.DataFrame()
pd_friends_1[['user1_id','user2_id']] = pd_friends[['user2_id','user1_id']]
exclusions = pd.concat([ pd_friends, pd_friends_1] , sort = False)
exclusions['friend_key'] = [k for k in map(concat, exclusions.user1_id, exclusions.user2_id)]
pd_rec = pd_rec[['user_id_1','user_id_2']][ ~pd_rec.friend_key.isin(exclusions.friend_key)  ]
pd_rec.columns = ['user_id', 'recommended_id']
pd_rec.reset_index(inplace=True)
pd_rec.drop(columns='index', inplace=True)

In [32]:
pd_rec

,user_id,recommended_id
0,1,3
1,2,3
2,3,1
3,3,2


In [33]:
pd_rec.equals(pd_answer_sql)

True